In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('Downloads\project\climate')


In [2]:
cl=pd.read_csv('climate2019.csv')

C:\Users\mohit\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cl=cl.drop([cl.index[270073],cl.index[76153],cl.index[162284],cl.index[215884]])

In [4]:
for i in range(3,10):
    cl.iloc[:,i]=cl.iloc[:,i].astype('float')

In [5]:
cl=cl.drop('Mandal',axis=1).groupby(['District','Date'],as_index=False).mean()

In [6]:
dat=[]
for date in cl['Date']:
    date = date[:10] 
    date = date.split('-')
    if(int(date[1])>9 and int(date[1])<13):
        da=date[2]+'-'+date[1]+'-'+date[0]
        
    else:
        da=date[0]+'-'+date[1]+'-'+date[2]
    dat.append(da)
cl['Date2']=dat        

In [7]:
len(set(cl['Date']))

364

In [8]:
"""import datetime
cl['Date2']=pd.to_datetime(cl['Date2'])

pd.date_range(start = '2019-01-01', end = '2019-12-31' ).difference(cl['Date2'])

len(set(cl['District']))"""

"import datetime\ncl['Date2']=pd.to_datetime(cl['Date2'])\n\npd.date_range(start = '2019-01-01', end = '2019-12-31' ).difference(cl['Date2'])\n\nlen(set(cl['District']))"

In [9]:
i=cl[(cl.District=='Mulug') | (cl.District=='Narayanpet')].index

In [10]:
cl=cl.drop(i)

In [11]:
cl=cl.replace('Warangal (U)','Warangal Urban')

In [12]:
cl=cl.replace('Warangal (R)','Warangal Rural')

In [13]:
cl=cl.replace('Kumuram Bheem - Asifabad','Komaram Bheem-Asifabad')

In [14]:
len(set(cl['District']))

31

In [15]:
lat=pd.DataFrame(set(cl['District']))

In [16]:
cl=cl.drop('Date',axis=1)

In [17]:
from keras.models import load_model
mod=load_model('../solar_rad_X_50d5_Y_38.h5')

Using TensorFlow backend.


In [18]:
cl=cl.rename(columns={'Rainfall (mm)': 'rain (mm)', 'temp_max (⁰C)': 'maxt (oC)', 'temp_min (⁰C)' : 'mint (oC)'})

In [19]:
cl['year ()']=2019

In [20]:
X=cl[['maxt (oC)','mint (oC)']]


X=X/[50,35]

y=mod.predict(X)

In [21]:
mod.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 15        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 58
Trainable params: 58
Non-trainable params: 0
_________________________________________________________________


In [22]:
y=y*38
y

cl['radn (MJ/m2)']=y

In [23]:
cl2=cl.drop(['Humidity_max (%)','humidity_min (%)', 'wind_speed_min (Kmph)', 'wind_speed_max (Kmph)'],axis=1)

In [24]:
cl_end=['Date2']
cl2=cl2[[c for c in cl2 if c not in cl_end]+cl_end]

In [25]:
cl3=cl.copy()

In [26]:
cl=cl2.copy()
cl['Day']=0
cl

,District,rain (mm),mint (oC),maxt (oC),year (),radn (MJ/m2),Date2,Day
0,Adilabad,0.011111,23.069444,32.608333,2019,21.149967,2019-10-01,0
1,Adilabad,1.972222,23.672222,30.283333,2019,17.691462,2019-11-01,0
2,Adilabad,0.000000,16.894444,30.250000,2019,21.499868,2019-12-01,0
3,Adilabad,0.250000,23.661111,33.438889,2019,21.914776,2019-10-02,0
4,Adilabad,0.488889,23.411111,31.466667,2019,19.429241,2019-11-02,0
...,...,...,...,...,...,...,...,...
11705,Yadadri-Bhongir,4.417647,22.200000,30.982353,2019,19.469625,2019-10-30,0
11706,Yadadri-Bhongir,0.000000,19.147059,31.411765,2019,21.770239,2019-11-30,0
11707,Yadadri-Bhongir,0.000000,15.870588,28.697059,2019,20.010345,2019-12-30,0
11708,Yadadri-Bhongir,0.358824,22.194118,29.870588,2019,17.979435,2019-10-31,0


In [27]:
idx = cl.columns

In [28]:
idx

Index(['District', 'rain (mm)', 'mint (oC)', 'maxt (oC)', 'year ()',
       'radn (MJ/m2)', 'Date2', 'Day'],
      dtype='object')

In [29]:
df=pd.DataFrame(columns=idx)
df

,District,rain (mm),mint (oC),maxt (oC),year (),radn (MJ/m2),Date2,Day


In [30]:
#days for district
for i in set(cl['District']):
    d=cl[cl['District']==i]
    f1=d['Date2']=='2019-09-09'
    f2=d['Date2']=='2019-09-11'
    s=[]
    for p,q in zip(f1,f2):
        s.append(p or q)
    f3=d[s].iloc[:,1:6]
    
    mis=f3.mean(axis=0)
    
    
    mis['District']=i
    mis['Date2']='2019-09-10'
    mis['Day']=0
    
    #mis=mis.transpose()
    mis = pd.DataFrame(mis,index=idx)
    
    mis=mis.T
    
    d=pd.concat([mis,d],ignore_index=True)
    d=d.sort_values('Date2')
    
    d['Day'] = list(range(1,366))
    print(d.shape)
   
    df=pd.concat([df,d],ignore_index=True)
    
    
    


(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)
(365, 8)


In [31]:
#2018 data
w=pd.read_csv('2018_daily_complete_weather_data_0.csv')

In [32]:
w=w.drop(['location','row_id','humidity_min','humidity_max','wind_speed_min','wind_speed_max'],axis=1)
    

In [33]:
w=w.drop('mandal',axis=1).groupby(['district','odate'],as_index=False).mean()

In [34]:
w=w.replace('Kumuram Bheem - Asifabad','Komaram Bheem-Asifabad')

In [35]:
w=w.replace('Warangal (R)','Warangal Rural')

In [36]:
w=w.replace('Warangal (U)','Warangal Urban')

In [37]:
set(w['district'])==set(cl['District'])

True

In [38]:
for i in range(2,5):
    w.iloc[:,i]=w.iloc[:,i].astype('float')

In [39]:
dat=[]
for date in w['odate']:
    date = date.split('/')
  
    da='20'+date[2]+'-'+date[1]+'-'+date[0]
    dat.append(da)
w['Date2']=dat  

In [40]:
w=w.rename(columns={'district':'District','cumm_rainfall':'rain (mm)','temp_min':'mint (oC)', 'temp_max':'maxt (oC)'})

In [41]:
X=w[['maxt (oC)','mint (oC)']]


X=X/[50,35]

y=mod.predict(X)

In [42]:
y=y*38


w['radn (MJ/m2)']=y

In [43]:
w['Day']=0
for i in set(w['District']):

    d=w[w['District']==i]
    d=d.sort_values('Date2')
    d['Day'] = list(range(1,366))
    w.loc[w.District==i]=d
    
    


In [44]:
w['Year']=2018

In [45]:
lat.rename(columns={0:'District'})

,District
0,Mancherial
1,Karimnagar
2,Medchal-Malkajgiri
3,Hyderabad
4,Yadadri-Bhongir
5,Peddapalli
6,Mahabubnagar
7,Vikarabad
8,Rangareddy
9,Bhadradri-Kothagudem


In [46]:
for i in range(1,13):
    lat[str(i)]=0


In [47]:

mon=[]
for date in df['Date2']:
    date = date.split('-')
    mon.append(date[1])
df['mon']=mon
mon=[]
for date in w['Date2']:
    date = date.split('-')
    mon.append(date[1])
w['mon']=mon
   
    

In [48]:
df['avg']=df[['mint (oC)', 'maxt (oC)']].mean(axis=1)
w['avg']=w[['mint (oC)', 'maxt (oC)']].mean(axis=1)

In [49]:
l=list(lat.columns.values.tolist())
l.remove(0)



In [50]:
#calculating tav and amp
def tav(df):
    for i in set(df['District']):
          d=df[df['District']==i]
          x=(d[['avg','mon']].groupby('mon').mean())
          lat.loc[lat[0]==i,l]=list(x['avg'])
    lat['tav']=lat[l].mean(axis=1)
    lat['amp']=lat[l].max(axis=1)-lat[l].min(axis=1)  
    return lat

In [51]:
tav(w)
latw=lat.copy()

In [52]:
tav(df)
latdf=lat.copy()

In [53]:
latitu=pd.read_csv('lat.csv')

In [54]:
latitu=latitu[['0','1']]
latitu=latitu.rename(columns={'0':'District','1':'Latitude'})

In [55]:
latitu.to_csv(r'Latitudes.csv')

In [56]:
latdf=latdf.rename(columns={0:'District'})

In [57]:
newl = latitu.copy()
float(newl.loc[newl.District=='Medak',['Latitude']]['Latitude'])

18.045992000000002

In [58]:
debug = True

In [59]:
# creating met format for apsim
def debug_print(*args,**kwargs):
    global debug
    if debug :
        print(*args,**kwargs)
def generate_mets(df,temps,lats):
    dists = df['District'].unique()
    met_str = None
    with open("met_template.txt",'r') as f :
        met_str = f.read()
    for dist in dists :
        df2=df[df['District']==dist]
        debug_print("checking district : ",dist)
        with open("./mets/{}.met".format(dist),"w") as f :
            
            res = temps.loc[temps.District==dist,['tav','amp']]
            tav,amp = float(res['tav']),float(res['amp'])
            lat = float(lats.loc[lats.District==dist,['Latitude']]['Latitude'])
            f.write(met_str.format(tav,amp,latitude=lat)+'\n'+'\n')
            f.write(' year    day radn   maxt   mint    rain'+'\n')
            f.write(' ()      () (MJ/m2) (oC)   (oC)    (mm)')
            f.write('\n'+df2.iloc[:,1:].to_string(index=False,col_space=2,header=False))
   

In [60]:
df.columns
df=df.rename(columns={'Day': 'Day ()'})


In [61]:
w=w.rename(columns={'Day': 'day ()', 'Year': 'year ()'})

In [62]:
df=df.rename(columns={'Day ()': 'day ()'})

In [63]:
df.columns

Index(['District', 'rain (mm)', 'mint (oC)', 'maxt (oC)', 'year ()',
       'radn (MJ/m2)', 'Date2', 'day ()', 'mon', 'avg'],
      dtype='object')

In [64]:
w.columns

Index(['District', 'odate', 'rain (mm)', 'mint (oC)', 'maxt (oC)', 'Date2',
       'radn (MJ/m2)', 'day ()', 'year ()', 'mon', 'avg'],
      dtype='object')

In [65]:
cols =  [ 'District','year ()','day ()','radn (MJ/m2)', 'maxt (oC)','mint (oC)', 'rain (mm)'] 

In [66]:
df18=w[cols]
df19=df[cols]

In [67]:
new_df = pd.concat([df18,df19])

In [68]:
new_df.to_csv(r'NEWDF.csv')

In [69]:
print(str(new_df.head().__repr__()))

   District year () day () radn (MJ/m2) maxt (oC) mint (oC) rain (mm)
0  Adilabad    2018      1      24.6152   30.4368      11.2         0
1  Adilabad    2018     32      26.1248   33.6211   12.4263         0
2  Adilabad    2018     60      26.3976   37.0474   20.1316         0
3  Adilabad    2018     91       26.925   40.4316   26.1474         0
4  Adilabad    2018    121      27.6051   43.0842   29.1684         0


In [70]:
new_df=new_df.sort_values(['year ()','day ()'])

In [71]:
new_df

,District,year (),day (),radn (MJ/m2),maxt (oC),mint (oC),rain (mm)
0,Adilabad,2018,1,24.6152,30.4368,11.2,0
365,Bhadradri-Kothagudem,2018,1,23.5804,32.6846,18.7769,0
730,Hyderabad,2018,1,23.3033,31.9118,17.5412,0
1095,Jagtial,2018,1,24.7981,31.5931,13.2759,0
1460,Jangaon,2018,1,24.5786,32.125,15.245,0
...,...,...,...,...,...,...,...
9854,Jayashankar-Bhupalpally,2019,365,18.9289,29.147,18.8258,0
10219,Kamareddy,2019,365,17.2499,28.1394,19.4027,0
10584,Jangaon,2019,365,17.5959,28.2222,18.9875,0
10949,Warangal Urban,2019,365,17.7703,28.6591,19.7091,0


In [72]:
for i in range(3,7):
    new_df.iloc[:,i]=new_df.iloc[:,i].astype('float').round(2)

In [73]:
new_df

,District,year (),day (),radn (MJ/m2),maxt (oC),mint (oC),rain (mm)
0,Adilabad,2018,1,24.62,30.44,11.20,0.0
365,Bhadradri-Kothagudem,2018,1,23.58,32.68,18.78,0.0
730,Hyderabad,2018,1,23.30,31.91,17.54,0.0
1095,Jagtial,2018,1,24.80,31.59,13.28,0.0
1460,Jangaon,2018,1,24.58,32.13,15.24,0.0
...,...,...,...,...,...,...,...
9854,Jayashankar-Bhupalpally,2019,365,18.93,29.15,18.83,0.0
10219,Kamareddy,2019,365,17.25,28.14,19.40,0.0
10584,Jangaon,2019,365,17.60,28.22,18.99,0.0
10949,Warangal Urban,2019,365,17.77,28.66,19.71,0.0


In [74]:
generate_mets(new_df,latdf,newl)

checking district :  Adilabad
checking district :  Bhadradri-Kothagudem
checking district :  Hyderabad
checking district :  Jagtial
checking district :  Jangaon
checking district :  Jayashankar-Bhupalpally
checking district :  Jogulamba-Gadwal
checking district :  Kamareddy
checking district :  Karimnagar
checking district :  Khammam
checking district :  Komaram Bheem-Asifabad
checking district :  Mahabubabad
checking district :  Mahabubnagar
checking district :  Mancherial
checking district :  Medak
checking district :  Medchal-Malkajgiri
checking district :  Nagarkurnool
checking district :  Nalgonda
checking district :  Nirmal
checking district :  Nizamabad
checking district :  Peddapalli
checking district :  Rajanna-Siricilla
checking district :  Rangareddy
checking district :  Sangareddy
checking district :  Siddipet
checking district :  Suryapet
checking district :  Vikarabad
checking district :  Wanaparthy
checking district :  Warangal Rural
checking district :  Warangal Urban
c